# Dataset Creator

This code will pull the data instances used for the Xception model. Note that there are multiple ways to attain the same end dataset, this set of code functions of an excel that coresponds to a unique cell. This code requires the raw images (nuclei, cell) and masks (nuclei,cell). 

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
#import plotly.graph_objects as go
import os
from scipy import stats
#from plotly.subplots import make_subplots
import os
from glob import glob
import cv2
import sklearn.model_selection as crv


In [ ]:
data=pd.read_csv('\\path to raw morphological csv\\') #read in dataset excel. note thgat each row is a cell


dataset=data.index.tolist() # create a list of indexes 


In [ ]:
# function to find nested tiff for raw file in a folder, will use to pull the  raw images

def raw_tiff_path_finder(parent_path,tiff_name):
    for path, directories, files in os.walk(parent_path):
        if tiff_name in files:
            return(os.path.join(path, tiff_name))
    
    
    

In [ ]:

cells=np.empty((0,output_size,output_size, 3)) #list of cells to append to
labels=np.empty(0) #list of labels for each cell




for i in range(dataset):
    code=dataset[i]
    try:
        row = data.filter(items = [code], axis=0) # filter the cell of interest by the index 
        raw_path='\\path to raw morphological actin\\'

        # now we will  pull the phalloidin information from the filtered dataframe
        filename=row['FileName_Phalloidin'][0]+'f' #filename of the actin

        mask_path='\\path to cell mask \\'
        filepath=os.path.join(mask_path, filename)
        img_copy=cv2.imread(filepath,-1) # read the cell mask image

        identifier=row['pixel_coordinate'][0] #pixel value at location which we will use for classificatiomn, coordinates with mask value 

        img_copy[img_copy!=identifier]=[0] # make everything except mask 0
        img_copy[img_copy==identifier]=[255] # make the mask centered at 255 pixels
        img_copy=img_copy.astype(np.uint8)
        contours = cv2.findContours(img_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        for cntr in contours: # should only be one contour 
            x,y,w,h = cv2.boundingRect(cntr) # Bound a rectangle around the
        startx = (crop_size - w)//2 # get the bounding box crop for x and y
        starty = (crop_size - h)//2
        #now we will pull the raw phalloidin file 
        filepath_phalloidin=raw_tiff_path_finder(raw_path,row['FileName_Phalloidin'][0]) # find the raw actin
        raw_phalloidin=cv2.imread(filepath_phalloidin,-1) #read the raw actin image
        raw_phalloidin[img_copy==0]=[0] # zero everyithing in the raw actin image except the mas (i.e. move debris and other cells interfering)

        result_mask = np.zeros((crop_size,crop_size)) # initialize a mask
        result_mask[starty:starty+h,startx:startx+w] = raw_phalloidin[y:y+h,x:x+w] # pull the boxed raw cell image


        # now we will try the DAPI Analysis 
        mask_path='\\path to nuclei mask \\' #
        filename=row['FileName_DAPI'][0]+'f' # filename of the DAPI Mask
        filepath=os.path.join(mask_path, filename)
        img_2=cv2.imread(filepath,-1)
        img_2[img_2!=identifier]=[0] # make everything except mask 0
        img_2[img_2==identifier]=[255] # make the mask centered at 255 pixels

        #READ IN THE DAPI FOLDER 
        filepath_dapi=raw_tiff_path_finder(raw_path,row['FileName_DAPI'][0]) # find the raw DAPI
        raw_DAPI=cv2.imread(filepath_dapi,-1)#read the raw nuclei image
        raw_DAPI[img_2==0]=[0] # make everything except mask 0
        result_mask_2 = np.zeros((crop_size,crop_size))# initialize a mask
        result_mask_2[starty:starty+h,startx:startx+w] = raw_DAPI[y:y+h,x:x+w]# pull the boxed raw nuclei image

        #now we will normalize the images
        cell_max=np.max(result_mask)
        nuclei_max=np.max(result_mask_2)

        #if an image is blanck 
        if cell_max==0:
            print(cell_max)
            cell_max=1
            print(key+'_'+code)
        if nuclei_max==0:
            print(nuclei_max)
            nuclei_max=1


        result_mask=result_mask/cell_max *255 #normalzied raw, actin
        result_mask_2=result_mask_2/nuclei_max *255 # normalized raw, DAPI







    #NOW WE WILL CREATE A COMPOSITE IMAGE WITH BOTH RAW CHANNELS

        result_composite=result_mask.copy()
        result_composite[result_mask_2!=0]=result_mask_2[result_mask_2!=0]

        result_mask=np.array([result_mask,result_mask,result_mask]).T
        result_mask_2=np.array([result_mask_2,result_mask_2,result_mask_2]).T
        result_composite=np.array([result_composite,result_composite,result_composite]).T

        bin_size = crop_size // output_size

        result_mask = result_mask.reshape((output_size, bin_size, 
                               output_size, bin_size, 3)).max(3).max(1)


        result_mask_2 = result_mask_2.reshape((output_size, bin_size, 
                               output_size, bin_size, 3)).max(3).max(1)        

        result_composite = result_composite.reshape((output_size, bin_size, 
                               output_size, bin_size, 3)).max(3).max(1)


        result_mask[:,:,0]=result_mask_2[:,:,0]
        result_mask[:,:,2]=result_composite[:,:,0]


        result_mask=np.expand_dims(result_mask, axis=0)


        if cell_max>1: # append to a dataset
            cells=np.append(cells,result_mask,axis=0)
            labels=np.append(labels,code)

In [ ]:
np.save('\\path to save cell instances  \\', cells) #save the cells dataset
np.save('\\path to save labelks  \\', labels)#save the labels dataset